In [198]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
%matplotlib inline

In [2]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

In [149]:
pd.set_option('display.max_rows', 500)

In [280]:
bom = pd.read_csv('Data/bom.movie_gross.csv')
# tmdb = pd.read_csv('Data/tmdb.movies.csv')
tn = pd.read_csv('Data/tn.movie_budgets.csv')

In [281]:
# cleaning up an encoding issue for apostrophes
tn.movie = tn.movie.apply(lambda x: x.replace('â\x80\x99',"'"))

# converting the money columns, which have values stored as strings due to dollar-signs and commas, to numbers
def convert_currency_str_to_num(dataframe, column):
    dataframe[column] = dataframe[column].apply(lambda x: x.replace('$',''))
    dataframe[column] = dataframe[column].apply(lambda x: x.replace(',',''))
    dataframe[column] = dataframe[column].astype('float')

convert_currency_str_to_num(tn, 'production_budget')
convert_currency_str_to_num(tn, 'domestic_gross')
convert_currency_str_to_num(tn, 'worldwide_gross')

tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09


In [282]:
tn['year'] = tn.release_date.apply(lambda x: int(x[-4:]))

In [283]:
tn = tn.loc[tn.year >= 2010]

In [284]:
tn['release_date'] = pd.to_datetime(tn.release_date)

In [285]:
tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011
2,3,2019-06-07,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,2015
...,...,...,...,...,...,...,...
5761,62,2014-12-31,Stories of Our Lives,15000.0,0.0,0.000000e+00,2014
5771,72,2015-05-19,Family Motocross,10000.0,0.0,0.000000e+00,2015
5772,73,2012-01-13,Newlyweds,9000.0,4584.0,4.584000e+03,2012
5777,78,2018-12-31,Red 11,7000.0,0.0,0.000000e+00,2018


In [286]:
bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [287]:
# tmdb.head()

In [288]:
tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011
2,3,2019-06-07,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,2015


In [289]:
# find returns -1 if nothing found
# I did a bunch of poking around with the parentheses in the following cells, 
# and then combined everything into a a pair of functions

# bom['has_parentheses'] = bom.title.apply(lambda x: x.find("(") != -1)
# bom.loc[bom['has_parentheses']].info()

In [290]:
# bom['parenthetical'] = bom.title.apply(lambda x: x[(x.find("(")+1):(x.find(")"))] if x.find("(") != -1 else '')

In [291]:
# bom['parenthetical_is_year'] = bom.parenthetical.apply(lambda x: (x.isdigit()) and (len(x) == 4))

In [292]:
# bom['standardized_title'] = bom.title.loc[bom.parenthetical_is_year].apply(lambda x: x[:(x.find("(")-1)])

In [293]:
# bom['standardized_title'][1]

In [294]:
def parenthetical_is_a_year(words):
    if words.find("(") != -1:
        new_words = words[(words.find("(")+1):(words.find(")"))]
        if (new_words.isdigit()) and (len(new_words) == 4):
            return True
        else:
            return False
    else:
        return False

In [295]:
def remove_appended_yr_from_title(dataframe, column):
    dataframe[column] =  dataframe[column].apply(lambda x: x[:(x.find("(")-1)] if parenthetical_is_a_year(x) else x)

In [296]:
remove_appended_yr_from_title(bom, 'title')

In [297]:
bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland,BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [298]:
def title_string_format_standardization(dataframe, column):
    dataframe[column] = dataframe[column].apply(lambda x: str(x).translate(str.maketrans('', '', string.punctuation)).lower())

In [299]:
title_string_format_standardization(bom, 'title')

In [300]:
title_string_format_standardization(tn, 'movie')

In [301]:
bom.head(50)

,title,studio,domestic_gross,foreign_gross,year
0,toy story 3,BV,415000000.0,652000000,2010
1,alice in wonderland,BV,334200000.0,691300000,2010
2,harry potter and the deathly hallows part 1,WB,296000000.0,664300000,2010
3,inception,WB,292600000.0,535700000,2010
4,shrek forever after,P/DW,238700000.0,513900000,2010
5,the twilight saga eclipse,Sum.,300500000.0,398000000,2010
6,iron man 2,Par.,312400000.0,311500000,2010
7,tangled,BV,200800000.0,391000000,2010
8,despicable me,Uni.,251500000.0,291600000,2010
9,how to train your dragon,P/DW,217600000.0,277300000,2010


In [302]:
print(bom.columns)
# print(tmdb.columns)
print(tn.columns)
#only keeping certain columns, and dropping the 5 rows from bom that have nulls since the count is so low.
bom = bom[['title', 'studio']]
# tmdb = tmdb[['genre_ids', 'original_language', 'popularity', 
#              'release_date', 'title', 'vote_average', 'vote_count']]
tn = tn[['movie', 'release_date', 'production_budget', 'domestic_gross',
           'worldwide_gross']]
bom.dropna(inplace=True)
# recreate foreign gross

Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')
Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross', 'year'],
      dtype='object')


In [303]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3382 entries, 0 to 3386
Data columns (total 2 columns):
title     3382 non-null object
studio    3382 non-null object
dtypes: object(2)
memory usage: 79.3+ KB


In [304]:
len(bom.studio.unique())

257

In [305]:
# tmdb.info()

In [306]:
tn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2194 entries, 1 to 5780
Data columns (total 5 columns):
movie                2194 non-null object
release_date         2194 non-null datetime64[ns]
production_budget    2194 non-null float64
domestic_gross       2194 non-null float64
worldwide_gross      2194 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 102.8+ KB


In [307]:
# creating a foreign gross column
tn['foreign_gross'] = tn['worldwide_gross'] - tn['domestic_gross']
tn.head()

C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,movie,release_date,production_budget,domestic_gross,worldwide_gross,foreign_gross
1,pirates of the caribbean on stranger tides,2011-05-20,410600000.0,241063875.0,1.045664e+09,8.046000e+08
2,dark phoenix,2019-06-07,350000000.0,42762350.0,1.497624e+08,1.070000e+08
3,avengers age of ultron,2015-05-01,330600000.0,459005868.0,1.403014e+09,9.440081e+08
4,star wars ep viii the last jedi,2017-12-15,317000000.0,620181382.0,1.316722e+09,6.965404e+08
5,star wars ep vii the force awakens,2015-12-18,306000000.0,936662225.0,2.053311e+09,1.116649e+09


In [308]:
sum(bom.duplicated())

0

In [309]:
sum(tn.duplicated())

0

In [310]:
# sum(tmdb.duplicated())

In [311]:
# removing duplicates. This method keeps the first instance, but not duplicates.
# tmdb = tmdb.loc[tmdb.duplicated() == False]

# We ultimately decided this tmdb data was not important enough to include.
# We'll simply rely on the IMDb ratings

In [312]:
# bom['fuzzy_title'] = bom.title.apply(lambda x : [process.extract(x, tmdb.title, limit=1)][0][0][0])

In [313]:
# tn['fuzzy_title'] = tn.movie.apply(lambda x : [process.extract(x, tmdb.title, limit=1)][0][0][0])

In [314]:
studio_movie_performance_left = bom.merge(right=tn,
                                    how='left',
                                    left_on='title',
                                    right_on='movie')

In [315]:
sum(studio_movie_performance_left.movie.isna())

1939

In [316]:
studio_movie_performance_left.sort_values('title').head(100)
# .loc[studio_movie_performance_left.movie.isna()]

,title,studio,movie,release_date,production_budget,domestic_gross,worldwide_gross,foreign_gross
2394,10 cloverfield lane,Par.,10 cloverfield lane,2016-03-11,5000000.0,72082999.0,108286422.0,36203423.0
1038,10 years,Anch.,NaN,NaT,NaN,NaN,NaN,NaN
1830,1000 times good night,FM,NaN,NaT,NaN,NaN,NaN,NaN
2282,1001 grams,KL,NaN,NaT,NaN,NaN,NaN,NaN
3222,102 not out,Sony,NaN,NaT,NaN,NaN,NaN,NaN
533,111111,Rocket,NaN,NaT,NaN,NaN,NaN,NaN
3161,12 strong,WB,12 strong,2018-01-19,35000000.0,45819713.0,71118378.0,25298665.0
1168,12 years a slave,FoxS,12 years a slave,2013-10-18,20000000.0,56671993.0,181025343.0,124353350.0
94,127 hours,FoxS,127 hours,2010-11-05,18000000.0,18335230.0,60217171.0,41881941.0
486,13 assassins,Magn.,NaN,NaT,NaN,NaN,NaN,NaN


In [317]:
studio_movie_performance_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3387 entries, 0 to 3386
Data columns (total 8 columns):
title                3387 non-null object
studio               3387 non-null object
movie                1448 non-null object
release_date         1448 non-null datetime64[ns]
production_budget    1448 non-null float64
domestic_gross       1448 non-null float64
worldwide_gross      1448 non-null float64
foreign_gross        1448 non-null float64
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 238.1+ KB


In [318]:
studio_movie_performance_left.head()

,title,studio,movie,release_date,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,toy story 3,BV,toy story 3,2010-06-18,200000000.0,415004880.0,1.068880e+09,653874642.0
1,alice in wonderland,BV,alice in wonderland,2010-03-05,200000000.0,334191110.0,1.025491e+09,691300000.0
2,harry potter and the deathly hallows part 1,WB,NaN,NaT,NaN,NaN,NaN,NaN
3,inception,WB,inception,2010-07-16,160000000.0,292576195.0,8.355246e+08,542948447.0
4,shrek forever after,P/DW,shrek forever after,2010-05-21,165000000.0,238736787.0,7.562447e+08,517507886.0


In [319]:
studio_movie_performance_inner = bom.merge(right=tn,
                                    how='inner',
                                    left_on='title',
                                    right_on='movie')

In [320]:
studio_movie_performance_inner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1448 entries, 0 to 1447
Data columns (total 8 columns):
title                1448 non-null object
studio               1448 non-null object
movie                1448 non-null object
release_date         1448 non-null datetime64[ns]
production_budget    1448 non-null float64
domestic_gross       1448 non-null float64
worldwide_gross      1448 non-null float64
foreign_gross        1448 non-null float64
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 101.8+ KB


In [321]:
imdb_titles = pd.read_csv('Exploration/imdb_df_join3.csv')

In [322]:
imdb_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 10 columns):
tconst             146144 non-null object
primary_title      146144 non-null object
original_title     146123 non-null object
start_year         146144 non-null int64
runtime_minutes    114405 non-null float64
genres             146144 non-null object
averagerating      73856 non-null float64
numvotes           73856 non-null float64
directors          140417 non-null object
writers            110261 non-null object
dtypes: float64(3), int64(1), object(6)
memory usage: 11.2+ MB


In [323]:
imdb_titles.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,directors,writers
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,nm0712540,"nm0023551,nm1194313,nm0347899,nm1391276"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,nm0002411,NaN
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,nm0000080,"nm0000080,nm0462648"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,nm0611531,nm0347899
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"nm0765384,nm0749914","nm1360635,nm0749914"


In [324]:
len(set(imdb_titles.original_title))

137774

In [325]:
primary_titles = imdb_titles[['tconst','primary_title']]
original_titles = imdb_titles[['tconst','original_title']]
primary_titles.columns = ['tconst','title']
original_titles.columns = ['tconst','title']

In [326]:
imdb_titles_only = pd.concat([primary_titles, original_titles])

In [327]:
title_string_format_standardization(imdb_titles_only, 'title')

In [328]:
imdb_titles_only.head()

,tconst,title
0,tt0063540,sunghursh
1,tt0066787,one day before the rainy season
2,tt0069049,the other side of the wind
3,tt0069204,sabse bada sukh
4,tt0100275,the wandering soap opera


In [329]:
imdb_titles_only.tail()

,tconst,title
146139,tt9916538,kuambil lagi hatiku
146140,tt9916622,rodolpho teóphilo o legado de um pioneiro
146141,tt9916706,dankyavar danka
146142,tt9916730,6 gunn
146143,tt9916754,chico albuquerque revelações


In [330]:
imdb_titles_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292288 entries, 0 to 146143
Data columns (total 2 columns):
tconst    292288 non-null object
title     292288 non-null object
dtypes: object(2)
memory usage: 6.7+ MB


In [331]:
imdb_titles_only = imdb_titles_only.loc[imdb_titles_only.duplicated() == False]
imdb_titles_only.dropna(inplace=False)

,tconst,title
0,tt0063540,sunghursh
1,tt0066787,one day before the rainy season
2,tt0069049,the other side of the wind
3,tt0069204,sabse bada sukh
4,tt0100275,the wandering soap opera
...,...,...
146026,tt9899938,kibaiyanse watashi
146028,tt9900060,lupin the iiird mine fujiko no uso
146037,tt9900688,da san yuan
146121,tt9914254,kirsebæreventyret


In [332]:
imdb_titles_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160557 entries, 0 to 146135
Data columns (total 2 columns):
tconst    160557 non-null object
title     160557 non-null object
dtypes: object(2)
memory usage: 3.7+ MB


In [333]:
imdb_titles_only.reset_index(inplace=True)

In [334]:
imdb_titles_only.head()

,index,tconst,title
0,0,tt0063540,sunghursh
1,1,tt0066787,one day before the rainy season
2,2,tt0069049,the other side of the wind
3,3,tt0069204,sabse bada sukh
4,4,tt0100275,the wandering soap opera


In [335]:
imdb_titles_only.drop('index', axis=1, inplace=True)
imdb_titles_only.head()

,tconst,title
0,tt0063540,sunghursh
1,tt0066787,one day before the rainy season
2,tt0069049,the other side of the wind
3,tt0069204,sabse bada sukh
4,tt0100275,the wandering soap opera


In [336]:
studio_movie_performance_inner.head()

,title,studio,movie,release_date,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,toy story 3,BV,toy story 3,2010-06-18,200000000.0,415004880.0,1.068880e+09,653874642.0
1,alice in wonderland,BV,alice in wonderland,2010-03-05,200000000.0,334191110.0,1.025491e+09,691300000.0
2,inception,WB,inception,2010-07-16,160000000.0,292576195.0,8.355246e+08,542948447.0
3,shrek forever after,P/DW,shrek forever after,2010-05-21,165000000.0,238736787.0,7.562447e+08,517507886.0
4,the twilight saga eclipse,Sum.,the twilight saga eclipse,2010-06-30,68000000.0,300531751.0,7.061028e+08,405571077.0


In [337]:
studio_movie_performance_inner_w_imdb = studio_movie_performance_inner.merge(right=imdb_titles_only,
                                    how='inner',
                                    left_on='title',
                                    right_on='title')

In [338]:
studio_movie_performance_inner_w_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2077 entries, 0 to 2076
Data columns (total 9 columns):
title                2077 non-null object
studio               2077 non-null object
movie                2077 non-null object
release_date         2077 non-null datetime64[ns]
production_budget    2077 non-null float64
domestic_gross       2077 non-null float64
worldwide_gross      2077 non-null float64
foreign_gross        2077 non-null float64
tconst               2077 non-null object
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 162.3+ KB


In [339]:
studio_movie_performance_inner_w_imdb.head()

,title,studio,movie,release_date,production_budget,domestic_gross,worldwide_gross,foreign_gross,tconst
0,toy story 3,BV,toy story 3,2010-06-18,200000000.0,415004880.0,1.068880e+09,653874642.0,tt0435761
1,alice in wonderland,BV,alice in wonderland,2010-03-05,200000000.0,334191110.0,1.025491e+09,691300000.0,tt1014759
2,alice in wonderland,BV,alice in wonderland,2010-03-05,200000000.0,334191110.0,1.025491e+09,691300000.0,tt1926979
3,alice in wonderland,BV,alice in wonderland,2010-03-05,200000000.0,334191110.0,1.025491e+09,691300000.0,tt2049386
4,inception,WB,inception,2010-07-16,160000000.0,292576195.0,8.355246e+08,542948447.0,tt1375666


In [340]:
studio_movie_performance_inner_w_imdb.drop('movie', axis=1, inplace=True)

In [341]:
studio_movie_performance_inner_w_imdb.to_csv('Exploration/studio_movie_performance_inner_w_imdb.csv', index=False)

In [ ]:
# studio_movie_performance_left_w_imdb = studio_movie_performance_left.merge(right=tn,
#                                                                            how='left',
#                                                                            left_on='title',
#                                                                            right_on='movie')